In [1]:
#IMPORTS
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV #splits up your test set in eqally sized parts, uses one part as test data and the rest as training data. So it optimizes as many classifiers as parts you split your data into.

from sklearn.neighbors import KNeighborsClassifier


In [16]:
#COUNT UNIQUE USERS

df = pd.read_csv('USER-FEATURE-DATA.csv')

current_row=0
df_sample = pd.DataFrame([],columns=df.columns)
counter = 0
for i in np.unique(df['user1']):
      counter +=1

print(counter) 

20


In [17]:
#LOAD DATA

df = pd.read_csv('NEW_SAMPLES.csv')

current_row=0
df_sample = pd.DataFrame([],columns=df.columns)
for i in np.unique(df['user1']):
      try:
            new_sample=df[df['user1']==i].sample(n=10)
            df_sample=pd.concat([df_sample,new_sample],axis=0)
      except:
            pass

df_sample = df_sample.sample(frac=1).reset_index(drop=True) #Has 5 data point per user

y = df_sample.iloc[0:, 0]
X = df_sample.iloc[0:, 1:]
print(y) 

0       user3
1      user19
2      user29
3       user4
4      user44
        ...  
195    user48
196    user19
197    user49
198    user46
199    user43
Name: user1, Length: 200, dtype: object


In [14]:
#K NEAREST NEIGHBOURS
clf = KNeighborsClassifier(n_neighbors=1,algorithm='auto', metric='euclidean')

kf = KFold(n_splits=5)

for train_index, test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

    clf.fit(X_train, y_train)
    print('SCORE:', clf.score(X_test, y_test))

SCORE: 0.625
SCORE: 0.6
SCORE: 0.625
SCORE: 0.65
SCORE: 0.65


In [15]:
#TRAINING

kf = KFold(n_splits=5)

ATTRIBUTES = len(X.columns)
print('n attrb =', ATTRIBUTES)
USERS = y.nunique()
print('n users =', USERS)
neurons = (ATTRIBUTES + USERS) // 2 
clf = MLPClassifier(hidden_layer_sizes=(neurons,), solver='lbfgs', momentum=0.2, learning_rate_init=0.3, random_state=0)

clf.max_iter = 10000

clf.n_layers_ = 5

#got rid of:
#hidden_layer_sizes=(1,) (scores all zeroes)
#  l
#
# look into RandomSearchCV
param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd'],
            'hidden_layer_sizes': [
              (neurons-10,), (neurons-5,), (neurons,), (neurons+5,), (neurons+10,)
             ],
             'momentum': [0.1,0.2,0.3],
             'learning_rate_init':[0.2,0.3,0.4]
        }
       ]

GRID_SEARCH_TEST = False 

for train_index, test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

    if GRID_SEARCH_TEST:
        clf = GridSearchCV(MLPClassifier(), param_grid, cv=3, scoring='accuracy')
        clf.fit(X_train, y_train)
        print('params:', clf.best_params_)
    else:
      clf.n_layers_ = 5
      clf.fit(X_train, y_train)

      print('SCORE:', clf.score(X_test, y_test))
    
    
    #EXTRA INFO:
     

      #print('current loss computed with the loss function: ',clf.loss_)
      #print('coefs: ', clf.coefs_)
      #print('intercepts: ',clf.intercepts_)
      print(' number of iterations the solver: ', clf.n_iter_)
      print('num of layers: ', clf.n_layers_)
      #print('Num of o/p: ', clf.n_outputs_) 
      
    



n attrb = 102
n users = 20
SCORE: 0.75
 number of iterations the solver:  599
num of layers:  3
SCORE: 0.725
 number of iterations the solver:  824
num of layers:  3
SCORE: 0.75
 number of iterations the solver:  475
num of layers:  3
SCORE: 0.8
 number of iterations the solver:  924
num of layers:  3
SCORE: 0.725
 number of iterations the solver:  962
num of layers:  3
